In [2]:
# импорт необходимых библеотек
from pony.orm import *
import getpass
import os
from datetime import date
from models import db

# Исследование директории хранения данных

In [3]:
# вывод файлов директории 'data'
os.listdir('data')

['.ipynb_checkpoints',
 'imdb.vocab',
 'imdbEr.txt',
 'README.md',
 'sentiments.csv',
 'test',
 'train']

In [4]:
# вывод файлов директории 'data/train'
os.listdir('data/train')

['labeledBow.feat',
 'neg',
 'pos',
 'unsup',
 'unsupBow.feat',
 'urls_neg.txt',
 'urls_pos.txt',
 'urls_unsup.txt']

In [5]:
# вывод файлов директории 'data/train/pos'
os.listdir('data/train/pos')[:6]

['0_9.txt',
 '10000_8.txt',
 '10001_10.txt',
 '10002_7.txt',
 '10003_8.txt',
 '10004_8.txt']

In [6]:
# вывод файлов директории 'data/test'
os.listdir('data/test')

['labeledBow.feat', 'neg', 'pos', 'urls_neg.txt', 'urls_pos.txt']

In [7]:
# задание путей к файлам
PACH_DATA_TRAIN_POS = 'data/train/pos/'
PACH_DATA_TRAIN_NEG = 'data/train/neg/'
PACH_DATA_TRAIN_UNSUP = 'data/train/unsup/'
PACH_DATA_TEST_POS = 'data/test/pos/'
PACH_DATA_TEST_NEG = 'data/test/neg/'
PACH_DATA_TRAIN_URLS_POS = 'data/train/urls_pos.txt'
PACH_DATA_TRAIN_URLS_NEG = 'data/train/urls_neg.txt'
PACH_DATA_TRAIN_URLS_UNSUP = 'data/train/urls_unsup.txt'
PACH_DATA_TEST_URLS_POS = 'data/test/urls_pos.txt'
PACH_DATA_TEST_URLS_NEG = 'data/test/urls_neg.txt'

# Запись данных в базу данных Sentiment

[Модель данных](https://editor.ponyorm.com/user/RusLat/sentiment/designer)

<img src='images/sentiment.png'>

## Таблица 'Data' - тип выборки

In [8]:
# функция записи данных в таблицу 'Data'
@db_session
def add_data(data_name):
    db.Data(data_name=data_name)
    # commit() will be done automatically
    # database session cache will be cleared automatically
    # database connection will be returned to the pool

In [9]:
# данные для внесения данных в таблицу 'Data'
data_names = ['train', 'test', 'new']

In [10]:
# запись данных в таблицу 'Data'
for data_name in data_names:
    add_data(data_name)

In [11]:
# проверка записи данных
db.Data.select().show()

id|data_name
--+---------
2 |train    
3 |test     
4 |new      


Данные записаны **успешно**

## Таблица 'Label' - тип комментариев (отзывов)

In [12]:
# функция записи данных в таблицу 'Label'
@db_session
def add_label(data_name):
    db.Label(label_name=label_name)
    # commit() will be done automatically
    # database session cache will be cleared automatically
    # database connection will be returned to the pool

In [13]:
# данные для внесения данных в таблицу 'Label'
label_names = ['pos', 'neg', 'unsup']

In [14]:
# запись данных в таблицу 'Label'
for label_name in label_names:
    add_label(label_name)

In [15]:
# проверка записи данных
db.Label.select().show()

id|label_name
--+----------
1 |pos       
2 |neg       
3 |unsup     


Данные записаны **успешно**

## Таблица 'Title' - url фильмов

In [16]:
# функция записи данных в таблицу 'Title'
@db_session
def add_title(title_url):
    db.Title(title_url=title_url)
    # commit() will be done automatically
    # database session cache will be cleared automatically
    # database connection will be returned to the pool

In [17]:
# список путей к файлам с url фильмов
PACH_DATA_URLS = [PACH_DATA_TRAIN_URLS_POS, PACH_DATA_TRAIN_URLS_NEG, PACH_DATA_TRAIN_URLS_UNSUP, \
                  PACH_DATA_TEST_URLS_POS, PACH_DATA_TEST_URLS_NEG]

In [18]:
%%time
# объдинение путей к файлам в единый список
data_urls = []
for PACH_DATA_URL in PACH_DATA_URLS:
    with open(PACH_DATA_URL, 'r') as f:
        urls = f.read().split()
        data_urls += urls

CPU times: total: 46.9 ms
Wall time: 38 ms


In [19]:
# проверка соответсвия данных
len(data_urls), len(set(data_urls))

(100000, 14127)

In [20]:
# извлечение уникальных url фильмов и преобразование в список
data_urls = list(set(data_urls))
data_urls[:7]

['http://www.imdb.com/title/tt0114759/usercomments',
 'http://www.imdb.com/title/tt0050212/usercomments',
 'http://www.imdb.com/title/tt0113313/usercomments',
 'http://www.imdb.com/title/tt0343601/usercomments',
 'http://www.imdb.com/title/tt0363056/usercomments',
 'http://www.imdb.com/title/tt0374298/usercomments',
 'http://www.imdb.com/title/tt0081070/usercomments']

In [25]:
%%time
# запись данных в таблицу 'Title'
for title_url in data_urls:
    add_title(title_url)    

CPU times: total: 0 ns
Wall time: 0 ns


In [23]:
# проверка записи данных
select(count(t) for t in db.Title).show()

count(t)
--------
14127   


In [24]:
# проверка записи данных
db.Title.select()[:10].show()

id   |title_url                                       
-----+------------------------------------------------
12776|http://www.imdb.com/title/tt0272022/usercomments
4582 |http://www.imdb.com/title/tt0093582/usercomments
872  |http://www.imdb.com/title/tt0133832/usercomments
9927 |http://www.imdb.com/title/tt0114108/usercomments
2102 |http://www.imdb.com/title/tt0119423/usercomments
6363 |http://www.imdb.com/title/tt0380066/usercomments
9394 |http://www.imdb.com/title/tt0397150/usercomments
13428|http://www.imdb.com/title/tt0050381/usercomments
3175 |http://www.imdb.com/title/tt0449662/usercomments
8650 |http://www.imdb.com/title/tt0844330/usercomments


Данные записаны **успешно**

## Таблица 'Sentiment' - комментарии (отзывы) пользователей

In [26]:
# функция записи данных в таблицу 'Sentiment'
@db_session
def add_sentiment(rating, review, label_id, data_id, title_id):
    db.Sentiment(rating=rating, review=review, \
              label_id=label_id, data_id=data_id, title_id=title_id)
    # commit() will be done automatically
    # database session cache will be cleared automatically
    # database connection will be returned to the pool

In [27]:
# список путей к файлам с комментариями (озывами)
PACH_DATA = [PACH_DATA_TRAIN_POS, PACH_DATA_TRAIN_NEG, PACH_DATA_TRAIN_UNSUP, \
             PACH_DATA_TEST_POS, PACH_DATA_TEST_NEG]

In [28]:
# список путей к файлам с url фильмов
PACH_DATA_URLS

['data/train/urls_pos.txt',
 'data/train/urls_neg.txt',
 'data/train/urls_unsup.txt',
 'data/test/urls_pos.txt',
 'data/test/urls_neg.txt']

In [29]:
%%time
# запись данных в таблицу 'Sentiment'
for PACH, PACH_URL in zip(PACH_DATA, PACH_DATA_URLS):
    data_name = PACH.split('/')[1]
    label_name = PACH.split('/')[-2]
    data_id = db.Data.get(data_name=data_name).id
    label_id = db.Label.get(label_name=label_name).id
    with open(PACH_URL, 'r') as f:
        urls = f.read().split()
    DATA_PACH = os.listdir(PACH)    
    for filename, url in zip(DATA_PACH, urls):
        with open(os.path.join(PACH, filename), 'r', encoding="utf-8") as f:
            review = f.read()
        #sentiment_id = filename.split('.')[0].split('_')[0]
        rating = filename.split('.')[0].split('_')[1]
        title_id = db.Title.get(title_url=url).id
        
        add_sentiment(rating, review, label_id, data_id, title_id)

CPU times: total: 1min 6s
Wall time: 2min 9s


In [30]:
# проверка записи данных
select(count(s) for s in db.Sentiment).show()

count(s)
--------
100000  


In [31]:
# проверка записи данных
db.Sentiment.select()[:10].show()

sentiment_id|rating|review                        |label_id|data_id|title_id    
------------+------+------------------------------+--------+-------+------------
1           |9     |Bromwell High is a cartoon ...|Label[1]|Data[2]|Title[3876] 
2           |8     |Homelessness (or Houselessn...|Label[1]|Data[2]|Title[3876] 
3           |10    |Brilliant over-acting by Le...|Label[1]|Data[2]|Title[3876] 
4           |7     |This is easily the most und...|Label[1]|Data[2]|Title[93]   
5           |8     |This is not the typical Mel...|Label[1]|Data[2]|Title[93]   
6           |8     |This isn't the comedic Robi...|Label[1]|Data[2]|Title[3178] 
7           |7     |Yes its an art... to succes...|Label[1]|Data[2]|Title[3178] 
8           |7     |In this "critically acclaim...|Label[1]|Data[2]|Title[3178] 
14          |9     |I enjoyed The Night Listene...|Label[1]|Data[2]|Title[3178] 
1562        |8     |Parsifal (1982) Starring Mi...|Label[1]|Data[2]|Title[10661]


Данные записаны **успешно**